## Prerequirement
### 1. Download data
Dataset is derived from Fannie Mae’s [Single-Family Loan Performance Data](http://www.fanniemae.com/portal/funding-the-market/data/loan-performance-data.html) with all rights reserved by Fannie Mae. Refer to these [instructions](https://github.com/NVIDIA/spark-rapids-examples/blob/branch-22.08/docs/get-started/xgboost-examples/dataset/mortgage.md) to download the dataset.

### 2. Set up Environments
This notebook runs in a Dataproc cluster with GPU nodes, with [Spark RAPIDS](https://github.com/GoogleCloudDataproc/initialization-actions/tree/master/rapids) set up.

### 3. Start Jupyter Notebook 
```
$ jupyter notebook --ip=0.0.0.0 --port=8124 --no-browser

```


In [1]:
import time
import os
from pyspark import broadcast
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window


## Create Spark Session

In [2]:
spark = (SparkSession
    .builder
    .appName("MortgageETL")
    .getOrCreate())

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/07/28 08:53:54 WARN org.apache.spark.resource.ResourceProfile: The executor resource config for resource: gpu was specified but no corresponding task resource request was specified.
22/07/28 08:53:55 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
22/07/28 08:53:55 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
22/07/28 08:53:55 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
22/07/28 08:53:55 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator
22/07/28 08:53:56 WARN com.nvidia.spark.rapids.RapidsPluginUtils: RAPIDS Accelerator 22.04.0 using cudf 22.04.0.
22/07/28 08:53:56 WARN com.nvidia.spark.rapids.RapidsPluginUtils: RAPIDS Accelerator is enabled, to disable GPU support set `spark.rapids.sql.enabled` to false.
22/07/28 08:54:09 WARN com.nvidia.spark.udf.Plugin: Installing rapids UDF c

## Function Define
### 1. Define the constants

* Define input file schema (Performance and Acquisition)

In [3]:
# File schema
_csv_raw_schema = StructType([
      StructField("reference_pool_id", StringType()),
      StructField("loan_id", LongType()),
      StructField("monthly_reporting_period", StringType()),
      StructField("orig_channel", StringType()),
      StructField("seller_name", StringType()),
      StructField("servicer", StringType()),
      StructField("master_servicer", StringType()),
      StructField("orig_interest_rate", DoubleType()),
      StructField("interest_rate", DoubleType()),
      StructField("orig_upb", DoubleType()),
      StructField("upb_at_issuance", StringType()),
      StructField("current_actual_upb", DoubleType()),
      StructField("orig_loan_term", IntegerType()),
      StructField("orig_date", StringType()),
      StructField("first_pay_date", StringType()),    
      StructField("loan_age", DoubleType()),
      StructField("remaining_months_to_legal_maturity", DoubleType()),
      StructField("adj_remaining_months_to_maturity", DoubleType()),
      StructField("maturity_date", StringType()),
      StructField("orig_ltv", DoubleType()),
      StructField("orig_cltv", DoubleType()),
      StructField("num_borrowers", DoubleType()),
      StructField("dti", DoubleType()),
      StructField("borrower_credit_score", DoubleType()),
      StructField("coborrow_credit_score", DoubleType()),
      StructField("first_home_buyer", StringType()),
      StructField("loan_purpose", StringType()),
      StructField("property_type", StringType()),
      StructField("num_units", IntegerType()),
      StructField("occupancy_status", StringType()),
      StructField("property_state", StringType()),
      StructField("msa", DoubleType()),
      StructField("zip", IntegerType()),
      StructField("mortgage_insurance_percent", DoubleType()),
      StructField("product_type", StringType()),
      StructField("prepayment_penalty_indicator", StringType()),
      StructField("interest_only_loan_indicator", StringType()),
      StructField("interest_only_first_principal_and_interest_payment_date", StringType()),
      StructField("months_to_amortization", StringType()),
      StructField("current_loan_delinquency_status", IntegerType()),
      StructField("loan_payment_history", StringType()),
      StructField("mod_flag", StringType()),
      StructField("mortgage_insurance_cancellation_indicator", StringType()),
      StructField("zero_balance_code", StringType()),
      StructField("zero_balance_effective_date", StringType()),
      StructField("upb_at_the_time_of_removal", StringType()),
      StructField("repurchase_date", StringType()),
      StructField("scheduled_principal_current", StringType()),
      StructField("total_principal_current", StringType()),
      StructField("unscheduled_principal_current", StringType()),
      StructField("last_paid_installment_date", StringType()),
      StructField("foreclosed_after", StringType()),
      StructField("disposition_date", StringType()),
      StructField("foreclosure_costs", DoubleType()),
      StructField("prop_preservation_and_repair_costs", DoubleType()),
      StructField("asset_recovery_costs", DoubleType()),
      StructField("misc_holding_expenses", DoubleType()),
      StructField("holding_taxes", DoubleType()),
      StructField("net_sale_proceeds", DoubleType()),
      StructField("credit_enhancement_proceeds", DoubleType()),
      StructField("repurchase_make_whole_proceeds", StringType()),
      StructField("other_foreclosure_proceeds", DoubleType()),
      StructField("non_interest_bearing_upb", DoubleType()),
      StructField("principal_forgiveness_upb", StringType()),
      StructField("original_list_start_date", StringType()),
      StructField("original_list_price", StringType()),
      StructField("current_list_start_date", StringType()),
      StructField("current_list_price", StringType()),
      StructField("borrower_credit_score_at_issuance", StringType()),
      StructField("co-borrower_credit_score_at_issuance", StringType()),
      StructField("borrower_credit_score_current", StringType()),
      StructField("co-Borrower_credit_score_current", StringType()),
      StructField("mortgage_insurance_type", DoubleType()),
      StructField("servicing_activity_indicator", StringType()),
      StructField("current_period_modification_loss_amount", StringType()),
      StructField("cumulative_modification_loss_amount", StringType()),
      StructField("current_period_credit_event_net_gain_or_loss", StringType()),
      StructField("cumulative_credit_event_net_gain_or_loss", StringType()),
      StructField("homeready_program_indicator", StringType()),
      StructField("foreclosure_principal_write_off_amount", StringType()),
      StructField("relocation_mortgage_indicator", StringType()),
      StructField("zero_balance_code_change_date", StringType()),
      StructField("loan_holdback_indicator", StringType()),
      StructField("loan_holdback_effective_date", StringType()),
      StructField("delinquent_accrued_interest", StringType()),
      StructField("property_valuation_method", StringType()),
      StructField("high_balance_loan_indicator", StringType()),
      StructField("arm_initial_fixed-rate_period_lt_5_yr_indicator", StringType()),
      StructField("arm_product_type", StringType()),
      StructField("initial_fixed-rate_period", StringType()),
      StructField("interest_rate_adjustment_frequency", StringType()),
      StructField("next_interest_rate_adjustment_date", StringType()),
      StructField("next_payment_change_date", StringType()),
      StructField("index", StringType()),
      StructField("arm_cap_structure", StringType()),
      StructField("initial_interest_rate_cap_up_percent", StringType()),
      StructField("periodic_interest_rate_cap_up_percent", StringType()),
      StructField("lifetime_interest_rate_cap_up_percent", StringType()),
      StructField("mortgage_margin", StringType()),
      StructField("arm_balloon_indicator", StringType()),
      StructField("arm_plan_number", StringType()),
      StructField("borrower_assistance_plan", StringType()),
      StructField("hltv_refinance_option_indicator", StringType()),
      StructField("deal_name", StringType()),
      StructField("repurchase_make_whole_proceeds_flag", StringType()),
      StructField("alternative_delinquency_resolution", StringType()),
      StructField("alternative_delinquency_resolution_count", StringType()),
      StructField("total_deferral_amount", StringType())
      ])

* Define seller name mapping

In [4]:
# name mappings
_name_mapping = [
        ("WITMER FUNDING, LLC", "Witmer"),
        ("WELLS FARGO CREDIT RISK TRANSFER SECURITIES TRUST 2015", "Wells Fargo"),
        ("WELLS FARGO BANK,  NA" , "Wells Fargo"),
        ("WELLS FARGO BANK, N.A." , "Wells Fargo"),
        ("WELLS FARGO BANK, NA" , "Wells Fargo"),
        ("USAA FEDERAL SAVINGS BANK" , "USAA"),
        ("UNITED SHORE FINANCIAL SERVICES, LLC D\\/B\\/A UNITED WHOLESALE MORTGAGE" , "United Seq(e"),
        ("U.S. BANK N.A." , "US Bank"),
        ("SUNTRUST MORTGAGE INC." , "Suntrust"),
        ("STONEGATE MORTGAGE CORPORATION" , "Stonegate Mortgage"),
        ("STEARNS LENDING, LLC" , "Stearns Lending"),
        ("STEARNS LENDING, INC." , "Stearns Lending"),
        ("SIERRA PACIFIC MORTGAGE COMPANY, INC." , "Sierra Pacific Mortgage"),
        ("REGIONS BANK" , "Regions"),
        ("RBC MORTGAGE COMPANY" , "RBC"),
        ("QUICKEN LOANS INC." , "Quicken Loans"),
        ("PULTE MORTGAGE, L.L.C." , "Pulte Mortgage"),
        ("PROVIDENT FUNDING ASSOCIATES, L.P." , "Provident Funding"),
        ("PROSPECT MORTGAGE, LLC" , "Prospect Mortgage"),
        ("PRINCIPAL RESIDENTIAL MORTGAGE CAPITAL RESOURCES, LLC" , "Principal Residential"),
        ("PNC BANK, N.A." , "PNC"),
        ("PMT CREDIT RISK TRANSFER TRUST 2015-2" , "PennyMac"),
        ("PHH MORTGAGE CORPORATION" , "PHH Mortgage"),
        ("PENNYMAC CORP." , "PennyMac"),
        ("PACIFIC UNION FINANCIAL, LLC" , "Other"),
        ("OTHER" , "Other"),
        ("NYCB MORTGAGE COMPANY, LLC" , "NYCB"),
        ("NEW YORK COMMUNITY BANK" , "NYCB"),
        ("NETBANK FUNDING SERVICES" , "Netbank"),
        ("NATIONSTAR MORTGAGE, LLC" , "Nationstar Mortgage"),
        ("METLIFE BANK, NA" , "Metlife"),
        ("LOANDEPOT.COM, LLC" , "LoanDepot.com"),
        ("J.P. MORGAN MADISON AVENUE SECURITIES TRUST, SERIES 2015-1" , "JP Morgan Chase"),
        ("J.P. MORGAN MADISON AVENUE SECURITIES TRUST, SERIES 2014-1" , "JP Morgan Chase"),
        ("JPMORGAN CHASE BANK, NATIONAL ASSOCIATION" , "JP Morgan Chase"),
        ("JPMORGAN CHASE BANK, NA" , "JP Morgan Chase"),
        ("JP MORGAN CHASE BANK, NA" , "JP Morgan Chase"),
        ("IRWIN MORTGAGE, CORPORATION" , "Irwin Mortgage"),
        ("IMPAC MORTGAGE CORP." , "Impac Mortgage"),
        ("HSBC BANK USA, NATIONAL ASSOCIATION" , "HSBC"),
        ("HOMEWARD RESIDENTIAL, INC." , "Homeward Mortgage"),
        ("HOMESTREET BANK" , "Other"),
        ("HOMEBRIDGE FINANCIAL SERVICES, INC." , "HomeBridge"),
        ("HARWOOD STREET FUNDING I, LLC" , "Harwood Mortgage"),
        ("GUILD MORTGAGE COMPANY" , "Guild Mortgage"),
        ("GMAC MORTGAGE, LLC (USAA FEDERAL SAVINGS BANK)" , "GMAC"),
        ("GMAC MORTGAGE, LLC" , "GMAC"),
        ("GMAC (USAA)" , "GMAC"),
        ("FREMONT BANK" , "Fremont Bank"),
        ("FREEDOM MORTGAGE CORP." , "Freedom Mortgage"),
        ("FRANKLIN AMERICAN MORTGAGE COMPANY" , "Franklin America"),
        ("FLEET NATIONAL BANK" , "Fleet National"),
        ("FLAGSTAR CAPITAL MARKETS CORPORATION" , "Flagstar Bank"),
        ("FLAGSTAR BANK, FSB" , "Flagstar Bank"),
        ("FIRST TENNESSEE BANK NATIONAL ASSOCIATION" , "Other"),
        ("FIFTH THIRD BANK" , "Fifth Third Bank"),
        ("FEDERAL HOME LOAN BANK OF CHICAGO" , "Fedral Home of Chicago"),
        ("FDIC, RECEIVER, INDYMAC FEDERAL BANK FSB" , "FDIC"),
        ("DOWNEY SAVINGS AND LOAN ASSOCIATION, F.A." , "Downey Mortgage"),
        ("DITECH FINANCIAL LLC" , "Ditech"),
        ("CITIMORTGAGE, INC." , "Citi"),
        ("CHICAGO MORTGAGE SOLUTIONS DBA INTERFIRST MORTGAGE COMPANY" , "Chicago Mortgage"),
        ("CHICAGO MORTGAGE SOLUTIONS DBA INTERBANK MORTGAGE COMPANY" , "Chicago Mortgage"),
        ("CHASE HOME FINANCE, LLC" , "JP Morgan Chase"),
        ("CHASE HOME FINANCE FRANKLIN AMERICAN MORTGAGE COMPANY" , "JP Morgan Chase"),
        ("CHASE HOME FINANCE (CIE 1)" , "JP Morgan Chase"),
        ("CHASE HOME FINANCE" , "JP Morgan Chase"),
        ("CASHCALL, INC." , "CashCall"),
        ("CAPITAL ONE, NATIONAL ASSOCIATION" , "Capital One"),
        ("CALIBER HOME LOANS, INC." , "Caliber Funding"),
        ("BISHOPS GATE RESIDENTIAL MORTGAGE TRUST" , "Bishops Gate Mortgage"),
        ("BANK OF AMERICA, N.A." , "Bank of America"),
        ("AMTRUST BANK" , "AmTrust"),
        ("AMERISAVE MORTGAGE CORPORATION" , "Amerisave"),
        ("AMERIHOME MORTGAGE COMPANY, LLC" , "AmeriHome Mortgage"),
        ("ALLY BANK" , "Ally Bank"),
        ("ACADEMY MORTGAGE CORPORATION" , "Academy Mortgage"),
        ("NO CASH-OUT REFINANCE" , "OTHER REFINANCE"),
        ("REFINANCE - NOT SPECIFIED" , "OTHER REFINANCE"),
        ("Other REFINANCE" , "OTHER REFINANCE")]

* Define category (string) column and numeric column

In [5]:
# String columns
cate_col_names = [
        "orig_channel",
        "first_home_buyer",
        "loan_purpose",
        "property_type",
        "occupancy_status",
        "property_state",
        "product_type",
        "relocation_mortgage_indicator",
        "seller_name",
        "mod_flag"
]
# Numberic columns
label_col_name = "delinquency_12"
numeric_col_names = [
        "orig_interest_rate",
        "orig_upb",
        "orig_loan_term",
        "orig_ltv",
        "orig_cltv",
        "num_borrowers",
        "dti",
        "borrower_credit_score",
        "num_units",
        "zip",
        "mortgage_insurance_percent",
        "current_loan_delinquency_status",
        "current_actual_upb",
        "interest_rate",
        "loan_age",
        "msa",
        "non_interest_bearing_upb",
        label_col_name
]
all_col_names = cate_col_names + numeric_col_names

* Functions to extract perf and acq columns from raw schema

In [6]:
def extract_perf_columns(rawDf):
    perfDf = rawDf.select(
      col("loan_id"),
      date_format(to_date(col("monthly_reporting_period"),"MMyyyy"), "MM/dd/yyyy").alias("monthly_reporting_period"),
      upper(col("servicer")).alias("servicer"),
      col("interest_rate"),
      col("current_actual_upb"),
      col("loan_age"),
      col("remaining_months_to_legal_maturity"),
      col("adj_remaining_months_to_maturity"),
      date_format(to_date(col("maturity_date"),"MMyyyy"), "MM/yyyy").alias("maturity_date"),
      col("msa"),
      col("current_loan_delinquency_status"),
      col("mod_flag"),
      col("zero_balance_code"),
      date_format(to_date(col("zero_balance_effective_date"),"MMyyyy"), "MM/yyyy").alias("zero_balance_effective_date"),
      date_format(to_date(col("last_paid_installment_date"),"MMyyyy"), "MM/dd/yyyy").alias("last_paid_installment_date"),
      date_format(to_date(col("foreclosed_after"),"MMyyyy"), "MM/dd/yyyy").alias("foreclosed_after"),
      date_format(to_date(col("disposition_date"),"MMyyyy"), "MM/dd/yyyy").alias("disposition_date"),
      col("foreclosure_costs"),
      col("prop_preservation_and_repair_costs"),
      col("asset_recovery_costs"),
      col("misc_holding_expenses"),
      col("holding_taxes"),
      col("net_sale_proceeds"),
      col("credit_enhancement_proceeds"),
      col("repurchase_make_whole_proceeds"),
      col("other_foreclosure_proceeds"),
      col("non_interest_bearing_upb"),
      col("principal_forgiveness_upb"),
      col("repurchase_make_whole_proceeds_flag"),
      col("foreclosure_principal_write_off_amount"),
      col("servicing_activity_indicator"),
      col('quarter')
    )

    return perfDf.select("*").filter("current_actual_upb != 0.0")

def extract_acq_columns(rawDf):
    acqDf = rawDf.select(
      col("loan_id"),
      col("orig_channel"),
      upper(col("seller_name")).alias("seller_name"),
      col("orig_interest_rate"),
      col("orig_upb"),
      col("orig_loan_term"),
      date_format(to_date(col("orig_date"),"MMyyyy"), "MM/yyyy").alias("orig_date"),
      date_format(to_date(col("first_pay_date"),"MMyyyy"), "MM/yyyy").alias("first_pay_date"),
      col("orig_ltv"),
      col("orig_cltv"),
      col("num_borrowers"),
      col("dti"),
      col("borrower_credit_score"),
      col("first_home_buyer"),
      col("loan_purpose"),
      col("property_type"),
      col("num_units"),
      col("occupancy_status"),
      col("property_state"),
      col("zip"),
      col("mortgage_insurance_percent"),
      col("product_type"),
      col("coborrow_credit_score"),
      col("mortgage_insurance_type"),
      col("relocation_mortgage_indicator"),
      dense_rank().over(Window.partitionBy("loan_id").orderBy(to_date(col("monthly_reporting_period"),"MMyyyy"))).alias("rank"),
      col('quarter')
      )

    return acqDf.select("*").filter(col("rank")==1)

### 2. Define ETL Process

Define the function to do the ETL process

#### 2.1 Define Functions to Read Raw CSV File

* Define function to get quarter from input CSV file name

In [7]:
def _get_quarter_from_csv_file_name():
    return substring_index(substring_index(input_file_name(), '.', 1), '/', -1)

* Define function to read raw CSV data file

In [8]:
def read_raw_csv(spark, path):
    return spark.read.format('csv') \
            .option('nullValue', '') \
            .option('header', False) \
            .option('delimiter', '|') \
            .schema(_csv_raw_schema) \
            .load(path) \
            .withColumn('quarter', _get_quarter_from_csv_file_name())

#### 2.2 Define ETL Process

* Define function to parse dates in Performance data

In [9]:
def _parse_dates(perf):
    return perf \
            .withColumn('monthly_reporting_period', to_date(col('monthly_reporting_period'), 'MM/dd/yyyy')) \
            .withColumn('monthly_reporting_period_month', month(col('monthly_reporting_period'))) \
            .withColumn('monthly_reporting_period_year', year(col('monthly_reporting_period'))) \
            .withColumn('monthly_reporting_period_day', dayofmonth(col('monthly_reporting_period'))) \
            .withColumn('last_paid_installment_date', to_date(col('last_paid_installment_date'), 'MM/dd/yyyy')) \
            .withColumn('foreclosed_after', to_date(col('foreclosed_after'), 'MM/dd/yyyy')) \
            .withColumn('disposition_date', to_date(col('disposition_date'), 'MM/dd/yyyy')) \
            .withColumn('maturity_date', to_date(col('maturity_date'), 'MM/yyyy')) \
            .withColumn('zero_balance_effective_date', to_date(col('zero_balance_effective_date'), 'MM/yyyy'))

* Define function to create deliquency data frame from Performance data

In [10]:
def _create_perf_deliquency(spark, perf):
    aggDF = perf.select(
            col("quarter"),
            col("loan_id"),
            col("current_loan_delinquency_status"),
            when(col("current_loan_delinquency_status") >= 1, col("monthly_reporting_period")).alias("delinquency_30"),
            when(col("current_loan_delinquency_status") >= 3, col("monthly_reporting_period")).alias("delinquency_90"),
            when(col("current_loan_delinquency_status") >= 6, col("monthly_reporting_period")).alias("delinquency_180")) \
            .groupBy("quarter", "loan_id") \
            .agg(
                max("current_loan_delinquency_status").alias("delinquency_12"),
                min("delinquency_30").alias("delinquency_30"),
                min("delinquency_90").alias("delinquency_90"),
                min("delinquency_180").alias("delinquency_180")) \
            .select(
                col("quarter"),
                col("loan_id"),
                (col("delinquency_12") >= 1).alias("ever_30"),
                (col("delinquency_12") >= 3).alias("ever_90"),
                (col("delinquency_12") >= 6).alias("ever_180"),
                col("delinquency_30"),
                col("delinquency_90"),
                col("delinquency_180"))
    joinedDf = perf \
            .withColumnRenamed("monthly_reporting_period", "timestamp") \
            .withColumnRenamed("monthly_reporting_period_month", "timestamp_month") \
            .withColumnRenamed("monthly_reporting_period_year", "timestamp_year") \
            .withColumnRenamed("current_loan_delinquency_status", "delinquency_12") \
            .withColumnRenamed("current_actual_upb", "upb_12") \
            .select("quarter", "loan_id", "timestamp", "delinquency_12", "upb_12", "timestamp_month", "timestamp_year") \
            .join(aggDF, ["loan_id", "quarter"], "left_outer")

    # calculate the 12 month delinquency and upb values
    months = 12
    monthArray = [lit(x) for x in range(0, 12)]
    # explode on a small amount of data is actually slightly more efficient than a cross join
    testDf = joinedDf \
            .withColumn("month_y", explode(array(monthArray))) \
            .select(
                    col("quarter"),
                    floor(((col("timestamp_year") * 12 + col("timestamp_month")) - 24000) / months).alias("josh_mody"),
                    floor(((col("timestamp_year") * 12 + col("timestamp_month")) - 24000 - col("month_y")) / months).alias("josh_mody_n"),
                    col("ever_30"),
                    col("ever_90"),
                    col("ever_180"),
                    col("delinquency_30"),
                    col("delinquency_90"),
                    col("delinquency_180"),
                    col("loan_id"),
                    col("month_y"),
                    col("delinquency_12"),
                    col("upb_12")) \
            .groupBy("quarter", "loan_id", "josh_mody_n", "ever_30", "ever_90", "ever_180", "delinquency_30", "delinquency_90", "delinquency_180", "month_y") \
            .agg(max("delinquency_12").alias("delinquency_12"), min("upb_12").alias("upb_12")) \
            .withColumn("timestamp_year", floor((lit(24000) + (col("josh_mody_n") * lit(months)) + (col("month_y") - 1)) / lit(12))) \
            .selectExpr('*', 'pmod(24000 + (josh_mody_n * {}) + month_y, 12) as timestamp_month_tmp'.format(months)) \
            .withColumn("timestamp_month", when(col("timestamp_month_tmp") == lit(0), lit(12)).otherwise(col("timestamp_month_tmp"))) \
            .withColumn("delinquency_12", ((col("delinquency_12") > 3).cast("int") + (col("upb_12") == 0).cast("int")).alias("delinquency_12")) \
            .drop("timestamp_month_tmp", "josh_mody_n", "month_y")

    return perf.withColumnRenamed("monthly_reporting_period_month", "timestamp_month") \
            .withColumnRenamed("monthly_reporting_period_year", "timestamp_year") \
            .join(testDf, ["quarter", "loan_id", "timestamp_year", "timestamp_month"], "left") \
            .drop("timestamp_year", "timestamp_month")

* Define function to create acquisition data frame from Acquisition data

In [11]:
def _create_acquisition(spark, acq):
    nameMapping = spark.createDataFrame(_name_mapping, ["from_seller_name", "to_seller_name"])
    return acq.join(nameMapping, col("seller_name") == col("from_seller_name"), "left") \
      .drop("from_seller_name") \
      .withColumn("old_name", col("seller_name")) \
      .withColumn("seller_name", coalesce(col("to_seller_name"), col("seller_name"))) \
      .drop("to_seller_name") \
      .withColumn("orig_date", to_date(col("orig_date"), "MM/yyyy")) \
      .withColumn("first_pay_date", to_date(col("first_pay_date"), "MM/yyyy")) 

#### 2.3 Define Casting Process
This part is casting String column to Numbric. 
Example:
```
col_1
 "a"
 "b"
 "c"
 "a"
# After String ====> Numberic
col_1
 0
 1
 2
 0
```  
<br>

* Define function to get column dictionary

    Example
    ```
    col1 = [row(data="a",id=0), row(data="b",id=1)]
    ```

In [12]:
def _gen_dictionary(etl_df, col_names):
    cnt_table = etl_df.select(posexplode(array([col(i) for i in col_names])))\
                    .withColumnRenamed("pos", "column_id")\
                    .withColumnRenamed("col", "data")\
                    .filter("data is not null")\
                    .groupBy("column_id", "data")\
                    .count()
    windowed = Window.partitionBy("column_id").orderBy(desc("count"))
    return cnt_table.withColumn("id", row_number().over(windowed)).drop("count")

* Define function to convert string columns to numeric

In [13]:
def _cast_string_columns_to_numeric(spark, input_df):
    cached_dict_df = _gen_dictionary(input_df, cate_col_names).cache()
    output_df = input_df
    #  Generate the final table with all columns being numeric.
    for col_pos, col_name in enumerate(cate_col_names):
        col_dict_df = cached_dict_df.filter(col("column_id") == col_pos)\
                                    .drop("column_id")\
                                    .withColumnRenamed("data", col_name)
        
        output_df = output_df.join(broadcast(col_dict_df), col_name, "left")\
                        .drop(col_name)\
                        .withColumnRenamed("id", col_name)
    return output_df        

#### 2.4 Define Main Function
In this function:
1. Parse date in Performance data by calling _parse_dates (parsed_perf)
2. Create deliqency dataframe(perf_deliqency) form Performance data by calling _create_perf_deliquency
3. Create cleaned acquisition dataframe(cleaned_acq) from Acquisition data by calling _create_acquisition
4. Join deliqency dataframe(perf_deliqency) and cleaned acquisition dataframe(cleaned_acq), get clean_df
5. Cast String column to Numbric in clean_df by calling _cast_string_columns_to_numeric, get casted_clean_df
6. Return casted_clean_df as final result

In [14]:
def run_mortgage(spark, perf, acq):
    parsed_perf = _parse_dates(perf)
    perf_deliqency = _create_perf_deliquency(spark, parsed_perf)
    cleaned_acq = _create_acquisition(spark, acq)
    clean_df = perf_deliqency.join(cleaned_acq, ["loan_id", "quarter"], "inner").drop("quarter")
    casted_clean_df = _cast_string_columns_to_numeric(spark, clean_df)\
                    .select(all_col_names)\
                    .withColumn(label_col_name, when(col(label_col_name) > 0, 1).otherwise(0))\
                    .fillna(float(0))
    return casted_clean_df

## Script Settings

### 1. File Path Settings
* Define input file path

In [15]:
# You need to update them to your real paths!
dataRoot = os.getenv("DATA_ROOT", 'gs://your-bucket/your-paths')
orig_raw_path = dataRoot + '/'

* Define output folder path

In [16]:
output_path = dataRoot + '/output/data/'
output_path_train = dataRoot + '/output/train/'
output_path_eval = dataRoot + '/output/eval/'
save_train_eval_dataset = True

### 2. Common Spark Settings

In [17]:
spark.conf.set('spark.rapids.sql.explain', 'ALL')
spark.conf.set('spark.rapids.sql.batchSizeBytes', '512M')
spark.conf.set('spark.rapids.sql.reader.batchSizeBytes', '768M')

## Run Part

### Read Raw File

In [18]:
rawDf = read_raw_csv(spark, orig_raw_path)
acq = extract_acq_columns(rawDf)
perf = extract_perf_columns(rawDf)

### Run ETL
#### 1. Add additional Spark settings

In [19]:
# GPU run, set to true
spark.conf.set('spark.rapids.sql.enabled', 'true')
# CPU run, set to false
# spark.conf.set('spark.rapids.sql.enabled', 'false')
spark.conf.set('spark.sql.files.maxPartitionBytes', '1G')
spark.conf.set("spark.rapids.sql.hasNans", "false")
# use GPU to read CSV
spark.conf.set("spark.rapids.sql.csv.read.double.enabled", "true")

#### 2.Read Parquet File and Run ETL Process, Save the Result

In [20]:
start = time.time()

# run main function to process data
out = run_mortgage(spark, perf, acq)

# save processed data
out.write.parquet(output_path, mode='overwrite')

# save processed data
if save_train_eval_dataset:
    etlDf = spark.read.parquet(output_path)

    # split 80% for training, 20% for test
    splits = etlDf.randomSplit([0.8, 0.2])

    splits[0].write.format('parquet').save(output_path_train, mode="overwrite")
    splits[1].write.format('parquet').save(output_path_eval, mode="overwrite")

# print explain and time
print(out.explain())
end = time.time()
print(end - start)
spark.stop()

22/07/28 08:54:20 WARN com.nvidia.spark.rapids.GpuOverrides: 
*Exec <ProjectExec> will run on GPU
  *Exec <WindowExec> will run on GPU
    *Expression <Alias> row_number() windowspecdefinition(column_id#2031, count#2040L DESC NULLS LAST, specifiedwindowframe(RowFrame, unboundedpreceding$(), currentrow$())) AS id#2045 will run on GPU
      *Expression <WindowExpression> row_number() windowspecdefinition(column_id#2031, count#2040L DESC NULLS LAST, specifiedwindowframe(RowFrame, unboundedpreceding$(), currentrow$())) will run on GPU
        *Expression <RowNumber> row_number() will run on GPU
        *Expression <WindowSpecDefinition> windowspecdefinition(column_id#2031, count#2040L DESC NULLS LAST, specifiedwindowframe(RowFrame, unboundedpreceding$(), currentrow$())) will run on GPU
          *Expression <SortOrder> count#2040L DESC NULLS LAST will run on GPU
          *Expression <SpecifiedWindowFrame> specifiedwindowframe(RowFrame, unboundedpreceding$(), currentrow$()) will run on GPU

22/07/28 08:54:25 WARN com.nvidia.spark.rapids.GpuOverrides: 
*Exec <ProjectExec> will run on GPU
  *Expression <Alias> coalesce(orig_channel#2245, 0) AS orig_channel#4242 will run on GPU
    *Expression <Coalesce> coalesce(orig_channel#2245, 0) will run on GPU
  *Expression <Alias> coalesce(first_home_buyer#2450, 0) AS first_home_buyer#4243 will run on GPU
    *Expression <Coalesce> coalesce(first_home_buyer#2450, 0) will run on GPU
  *Expression <Alias> coalesce(loan_purpose#2655, 0) AS loan_purpose#4244 will run on GPU
    *Expression <Coalesce> coalesce(loan_purpose#2655, 0) will run on GPU
  *Expression <Alias> coalesce(property_type#2860, 0) AS property_type#4245 will run on GPU
    *Expression <Coalesce> coalesce(property_type#2860, 0) will run on GPU
  *Expression <Alias> coalesce(occupancy_status#3065, 0) AS occupancy_status#4246 will run on GPU
    *Expression <Coalesce> coalesce(occupancy_status#3065, 0) will run on GPU
  *Expression <Alias> coalesce(property_state#3270, 0) 

                                                              *Expression <EqualTo> (rank#331 = 1) will run on GPU
                                                            *Exec <WindowExec> will run on GPU
                                                              *Expression <Alias> dense_rank(_w1#342) windowspecdefinition(loan_id#1781L, _w1#342 ASC NULLS FIRST, specifiedwindowframe(RowFrame, unboundedpreceding$(), currentrow$())) AS rank#331 will run on GPU
                                                                *Expression <WindowExpression> dense_rank(_w1#342) windowspecdefinition(loan_id#1781L, _w1#342 ASC NULLS FIRST, specifiedwindowframe(RowFrame, unboundedpreceding$(), currentrow$())) will run on GPU
                                                                  *Expression <DenseRank> dense_rank(_w1#342) will run on GPU
                                                                  *Expression <WindowSpecDefinition> windowspecdefinition(loan_id#1781L, _w1#

22/07/28 08:54:26 WARN com.nvidia.spark.rapids.GpuOverrides: 
!Exec <ShuffleExchangeExec> cannot run on GPU because Columnar exchange without columnar children is inefficient
  @Partitioning <HashPartitioning> could run on GPU
    @Expression <AttributeReference> quarter#216 could run on GPU
    @Expression <AttributeReference> loan_id#1L could run on GPU
    @Expression <Cast> cast(timestamp_year#1311 as bigint) could run on GPU
      @Expression <AttributeReference> timestamp_year#1311 could run on GPU
    @Expression <Cast> cast(timestamp_month#1275 as bigint) could run on GPU
      @Expression <AttributeReference> timestamp_month#1275 could run on GPU
  !Exec <ProjectExec> cannot run on GPU because not all expressions can be replaced
    @Expression <AttributeReference> loan_id#1L could run on GPU
    @Expression <AttributeReference> interest_rate#8 could run on GPU
    @Expression <AttributeReference> current_actual_upb#11 could run on GPU
    @Expression <AttributeReference> loan

22/07/28 08:54:37 WARN com.nvidia.spark.rapids.GpuOverrides: >   (0 + 5) / 16]6]
*Exec <ProjectExec> will run on GPU
  *Expression <Alias> coalesce(orig_channel#2245, 0) AS orig_channel#4242 will run on GPU
    *Expression <Coalesce> coalesce(orig_channel#2245, 0) will run on GPU
  *Expression <Alias> coalesce(first_home_buyer#2450, 0) AS first_home_buyer#4243 will run on GPU
    *Expression <Coalesce> coalesce(first_home_buyer#2450, 0) will run on GPU
  *Expression <Alias> coalesce(loan_purpose#2655, 0) AS loan_purpose#4244 will run on GPU
    *Expression <Coalesce> coalesce(loan_purpose#2655, 0) will run on GPU
  *Expression <Alias> coalesce(property_type#2860, 0) AS property_type#4245 will run on GPU
    *Expression <Coalesce> coalesce(property_type#2860, 0) will run on GPU
  *Expression <Alias> coalesce(occupancy_status#3065, 0) AS occupancy_status#4246 will run on GPU
    *Expression <Coalesce> coalesce(occupancy_status#3065, 0) will run on GPU
  *Expression <Alias> coalesce(prope

...

22/07/28 08:55:07 WARN com.nvidia.spark.rapids.GpuOverrides:      (7 + 10) / 17]
*Exec <ProjectExec> will run on GPU
  *Expression <Alias> coalesce(orig_channel#2245, 0) AS orig_channel#4242 will run on GPU
    *Expression <Coalesce> coalesce(orig_channel#2245, 0) will run on GPU
  *Expression <Alias> coalesce(first_home_buyer#2450, 0) AS first_home_buyer#4243 will run on GPU
    *Expression <Coalesce> coalesce(first_home_buyer#2450, 0) will run on GPU
  *Expression <Alias> coalesce(loan_purpose#2655, 0) AS loan_purpose#4244 will run on GPU
    *Expression <Coalesce> coalesce(loan_purpose#2655, 0) will run on GPU
  *Expression <Alias> coalesce(property_type#2860, 0) AS property_type#4245 will run on GPU
    *Expression <Coalesce> coalesce(property_type#2860, 0) will run on GPU
  *Expression <Alias> coalesce(occupancy_status#3065, 0) AS occupancy_status#4246 will run on GPU
    *Expression <Coalesce> coalesce(occupancy_status#3065, 0) will run on GPU
  *Expression <Alias> coalesce(prope

22/07/28 08:55:13 WARN com.nvidia.spark.rapids.GpuOverrides:                    
*Exec <ProjectExec> will run on GPU
  *Expression <Alias> coalesce(orig_channel#2245, 0) AS orig_channel#4242 will run on GPU
    *Expression <Coalesce> coalesce(orig_channel#2245, 0) will run on GPU
  *Expression <Alias> coalesce(first_home_buyer#2450, 0) AS first_home_buyer#4243 will run on GPU
    *Expression <Coalesce> coalesce(first_home_buyer#2450, 0) will run on GPU
  *Expression <Alias> coalesce(loan_purpose#2655, 0) AS loan_purpose#4244 will run on GPU
    *Expression <Coalesce> coalesce(loan_purpose#2655, 0) will run on GPU
  *Expression <Alias> coalesce(property_type#2860, 0) AS property_type#4245 will run on GPU
    *Expression <Coalesce> coalesce(property_type#2860, 0) will run on GPU
  *Expression <Alias> coalesce(occupancy_status#3065, 0) AS occupancy_status#4246 will run on GPU
    *Expression <Coalesce> coalesce(occupancy_status#3065, 0) will run on GPU
  *Expression <Alias> coalesce(prope

22/07/28 08:55:14 WARN com.nvidia.spark.rapids.GpuOverrides: 
*Exec <ProjectExec> will run on GPU
  *Expression <Alias> coalesce(orig_channel#2245, 0) AS orig_channel#4242 will run on GPU
    *Expression <Coalesce> coalesce(orig_channel#2245, 0) will run on GPU
  *Expression <Alias> coalesce(first_home_buyer#2450, 0) AS first_home_buyer#4243 will run on GPU
    *Expression <Coalesce> coalesce(first_home_buyer#2450, 0) will run on GPU
  *Expression <Alias> coalesce(loan_purpose#2655, 0) AS loan_purpose#4244 will run on GPU
    *Expression <Coalesce> coalesce(loan_purpose#2655, 0) will run on GPU
  *Expression <Alias> coalesce(property_type#2860, 0) AS property_type#4245 will run on GPU
    *Expression <Coalesce> coalesce(property_type#2860, 0) will run on GPU
  *Expression <Alias> coalesce(occupancy_status#3065, 0) AS occupancy_status#4246 will run on GPU
    *Expression <Coalesce> coalesce(occupancy_status#3065, 0) will run on GPU
  *Expression <Alias> coalesce(property_state#3270, 0) 

22/07/28 08:55:22 WARN com.nvidia.spark.rapids.GpuOverrides:                    
*Exec <DataWritingCommandExec> will run on GPU
  *Output <InsertIntoHadoopFsRelationCommand> will run on GPU
  *Exec <SampleExec> will run on GPU
    *Exec <SortExec> will run on GPU
      *Expression <SortOrder> orig_channel#5321 ASC NULLS FIRST will run on GPU
      *Expression <SortOrder> first_home_buyer#5322 ASC NULLS FIRST will run on GPU
      *Expression <SortOrder> loan_purpose#5323 ASC NULLS FIRST will run on GPU
      *Expression <SortOrder> property_type#5324 ASC NULLS FIRST will run on GPU
      *Expression <SortOrder> occupancy_status#5325 ASC NULLS FIRST will run on GPU
      *Expression <SortOrder> property_state#5326 ASC NULLS FIRST will run on GPU
      *Expression <SortOrder> product_type#5327 ASC NULLS FIRST will run on GPU
      *Expression <SortOrder> relocation_mortgage_indicator#5328 ASC NULLS FIRST will run on GPU
      *Expression <SortOrder> seller_name#5329 ASC NULLS FIRST will 

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project [coalesce(orig_channel#2245, 0) AS orig_channel#4242, coalesce(first_home_buyer#2450, 0) AS first_home_buyer#4243, coalesce(loan_purpose#2655, 0) AS loan_purpose#4244, coalesce(property_type#2860, 0) AS property_type#4245, coalesce(occupancy_status#3065, 0) AS occupancy_status#4246, coalesce(property_state#3270, 0) AS property_state#4247, coalesce(product_type#3475, 0) AS product_type#4248, coalesce(relocation_mortgage_indicator#3680, 0) AS relocation_mortgage_indicator#4249, coalesce(seller_name#3885, 0) AS seller_name#4250, coalesce(id#2045, 0) AS mod_flag#4251, coalesce(nanvl(orig_interest_rate#1787, null), 0.0) AS orig_interest_rate#4252, coalesce(nanvl(orig_upb#1789, null), 0.0) AS orig_upb#4253, coalesce(orig_loan_term#1792, 0) AS orig_loan_term#4254, coalesce(nanvl(orig_ltv#1799, null), 0.0) AS orig_ltv#4255, coalesce(nanvl(orig_cltv#1800, null), 0.0) AS orig_cltv#4256, coalesce(nanvl(num_borrowers#1801, null), 0